https://wiki.fysik.dtu.dk/ase/ase/calculators/espresso.html

https://github.com/Yavar-Azar/Courses/tree/main/QE/ASE-QE

In [ ]:
# ! export ASE_ESPRESSO_COMMAND="/path/to/pw.x -in PREFIX.pwi > PREFIX.pwo"

In [ ]:
from ase.io import read, write
from ase.calculators.espresso import Espresso

In [ ]:
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

In [ ]:
from ase.visualize import view

view(crystal)

In [ ]:
# inp_data={'prefix':"myprefix",'electron_maxstep':200,'outdir':"./",        'pseudo_dir':"./",'tstress':True,'tprnfor':True,'calculation':'scf', 
#  'ecutrho':240,'verbosity':'high','ecutwfc':30, 'diagonalization': 'david', 'occupations':'smearing','smearing':'mp', 'mixing_mode':'plain', 'mixing_beta':0.7,'degauss':0.001, 'nspin':1}

# pseudodict={"Na":"", "As":"As.pbe-n-rrkjus_psl.1.0.0.UPF"}

In [ ]:
crystal.calc=Espresso(pseudopotentials=pseudodict,input_data=inp_data, kpts=(1,1,1))
crystal.get_potential_energy()

In [ ]:
write("input.pwi", crystal, pseudopotentials=pseudodict, input_data=inp_data, kpts=(1,1,1))

In [ ]:
# https://sites.google.com/site/dceresoli/pseudopotentials
pseudopotentials = {'Na': 'Na.pbe-tm-gipaw-dc.UPF',
                    'Cl': 'Cl.pbe-tm-gipaw.UPF',
                    'Cu': 'Cu.pbe-tm-new-gipaw.UPF',
                    'Sb': 'sb_pbe_v1.4.uspp.F.UPF', # copied from another folder
                    'N': 'N.pbe-tm-new-gipaw-dc.UPF',
                    'F': 'F.pbe-tm-new-gipaw-dc.UPF',
                    'Si': 'Si.pbe-tm-new-gipaw-dc.UPF',
                    'C': 'C.pbe-tm-new-gipaw-dc.UPF',
                    'H': 'H.pbe-tm-new-gipaw-dc.UPF'
                    }

In [ ]:
import os
os.chdir('../..')
path = os.getcwd()
print(path)


In [ ]:
os.environ["ASE_ESPRESSO_COMMAND"] = "mpirun --oversubscribe -np 5 pw.x -in PREFIX.pwi > PREFIX.pwo"
os.environ["OMP_NUM_THREADS"] = "1,1"

head_dir = os.path.abspath("")
print(head_dir)

In [ ]:
# !export ASE_ESPRESSO_COMMAND="mpirun -np 10 pw.x -in PREFIX.pwi > PREFIX.pwo"; 
# !echo $ASE_ESPRESSO_COMMAND

In [ ]:
!pkill -9 -f pw.x

In [ ]:
from ase import units


In [ ]:
%%time

from ase.io import read, write

from ase.build import bulk
from ase.calculators.espresso import Espresso
from ase.constraints import UnitCellFilter
from ase.optimize import LBFGS

import os
from datetime import datetime
# os.makedirs('test', exist_ok=True)
# os.chdir('test')

os.chdir(head_dir)
# crystal=read("test/MIN-167-350K-CuCbPyz.cif")
tms = read("test/tms.xyz")

crystal = tms
a = 20.0
crystal.set_cell([a, a, a])
crystal.set_cell([(a, 0, 0), (0, a, 0), (0, 0, a)])

dir_name = f'{datetime.now().strftime("%Y-%m-%d_%I %M-%S_%p")}'
dir_name = 'test/' + dir_name
os.makedirs(dir_name, exist_ok=False)
os.chdir(dir_name)


pseudo_dir = '../../Pseudopotentials'



# crystal = bulk('NaCl', crystalstructure='rocksalt', a=6.0)


# calc = Espresso(pseudopotentials=pseudopotentials, pseudo_dir = pseudo_dir,
#                 tstress=True, tprnfor=True, ecutwfc=10, kpts=(3, 3, 3))
# crystal.calc=Espresso(pseudopotentials=pseudodict,input_data=inp_data, kpts=(1,1,1))

calc = Espresso(pseudopotentials=pseudopotentials, pseudo_dir = pseudo_dir,
outdir = './outdir',  prefix = 'crystal', restart_mode = 'from_scratch',
                tstress=True, tprnfor=True, nosym=True, 
                ecutwfc=50, kpts=(1, 1, 1), 
                ecutrho = 500,
                occupations = 'smearing', smearing = 'gauss', degauss = 1.0e-2
                )

# write("input.pwi", crystal, pseudopotentials=pseudodict, input_data=inp_data, kpts=(1,1,1))

crystal.calc = calc

# ucf = UnitCellFilter(rocksalt)
# opt = LBFGS(ucf)
# opt.run(fmax=0.005)

# # cubic lattic constant
# print((8*rocksalt.get_volume()/len(rocksalt))**(1.0/3.0))

print(crystal.get_potential_energy())
# print(rocksalt.get_forces())


In [ ]:
# &inputgipaw
#     job = 'nmr'
#     prefix = 'TMS'
#     tmp_dir = './outdir/'
#     q_gipaw = 0.01
#     spline_ps = .true.
#     use_nmr_macroscopic_shape = .false.
# /

gipaw_input = """&inputgipaw
    job = 'nmr'
    prefix = 'crystal'
    tmp_dir = './outdir/'
    q_gipaw = 0.01
    spline_ps = .true.
    use_nmr_macroscopic_shape = .false.
/
"""

with open("espresso_gipaw.pwi", "w") as f:
#    lines = ["Adding lines\n", "writing into it \n", "written successfully\n" ]
   f.writelines(gipaw_input)
   f.close()

import subprocess
f = open("espresso_gipaw.pwo", "a")
subprocess.call(["mpi run -np 5", "-in ",  "espresso_gipaw.pwi"], stdout=f)

subprocess.run(["gipaw.x", "-in", "espresso_gipaw.pwi", ">", "espresso_gipaw.pwo"])

In [ ]:
import psutil
 
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

In [ ]:
os.chdir(head_dir)
crystal=read("test/MIN-167-350K-CuCbPyz.cif")




In [ ]:
view(tms)

In [ ]:
print( crystal.get_cell() )
print( crystal.get_pbc() )

crystal=read("test/MIN-167-350K-CuCbPyz.cif")
crystal.get_forces()

In [ ]:
view(crystal)